In [1]:
from lstm import LSTM_Model
from data import init_datasets

In [2]:
topic = 'wiki'
path = 'data/corpora/'

In [3]:
batch_size = 20
time_steps = 10
freq_threshold = 1
epochs = 2

In [5]:
from data import _init_corpora
train, valid, test, vocab = _init_corpora(path=path, topic=topic, freq_threshold=freq_threshold)

In [9]:
train_t = train.reshape(-1, 1)
valid_t = valid.reshape(-1, 1)
test_t = test.reshape(-1, 1)

In [35]:
test.shape

(237728,)

In [10]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

In [11]:
TEST = torch.LongTensor(test)
TEST_T = torch.LongTensor(test_t)

In [12]:
time_steps=10

In [13]:
TEST = torch.split(tensor=TEST, split_size_or_sections=time_steps)
TEST_T = torch.split(tensor=TEST_T, split_size_or_sections=time_steps)

In [17]:
TEST[:3]

(tensor([22468,  1419, 22468,  1419, 14173,  2341,  9468, 10537,    77, 25899]),
 tensor([ 2378, 26059,  1685,    80, 12467, 12169,  1430, 12068,  1423, 24769]),
 tensor([22522, 18627, 26054, 25899, 23485, 26054,  3950, 13465,   797,    80]))

In [18]:
TEST_T[:3]

(tensor([[22468],
         [ 1419],
         [22468],
         [ 1419],
         [14173],
         [ 2341],
         [ 9468],
         [10537],
         [   77],
         [25899]]),
 tensor([[ 2378],
         [26059],
         [ 1685],
         [   80],
         [12467],
         [12169],
         [ 1430],
         [12068],
         [ 1423],
         [24769]]),
 tensor([[22522],
         [18627],
         [26054],
         [25899],
         [23485],
         [26054],
         [ 3950],
         [13465],
         [  797],
         [   80]]))

In [20]:
TEST_seq = pad_sequence(TEST, batch_first=True, padding_value=0)

In [21]:
TEST_T_seq = pad_sequence(TEST_T, batch_first=True, padding_value=0)

In [23]:
TEST_seq[-3:]

tensor([[15502,  7717, 18542,  7105,    77, 26313, 12434, 23496,  2866, 17241],
        [10880, 10291,  9056,    77, 13514, 22559,  2150,    77,  9054, 25836],
        [   77,  9054, 19242,    77,  2378,  9056,  1419,    80,     0,     0]])

In [24]:
TEST_T_seq[-3:]

tensor([[[15502],
         [ 7717],
         [18542],
         [ 7105],
         [   77],
         [26313],
         [12434],
         [23496],
         [ 2866],
         [17241]],

        [[10880],
         [10291],
         [ 9056],
         [   77],
         [13514],
         [22559],
         [ 2150],
         [   77],
         [ 9054],
         [25836]],

        [[   77],
         [ 9054],
         [19242],
         [   77],
         [ 2378],
         [ 9056],
         [ 1419],
         [   80],
         [    0],
         [    0]]])

In [26]:
TEST_seq_i = TEST_seq.narrow_copy(1, 0, TEST_seq.shape[1] - 1)
TEST_seq_o = TEST_seq.narrow_copy(1, 1, TEST_seq.shape[1] - 1)

In [27]:
TEST_seq_i[-3:]

tensor([[15502,  7717, 18542,  7105,    77, 26313, 12434, 23496,  2866],
        [10880, 10291,  9056,    77, 13514, 22559,  2150,    77,  9054],
        [   77,  9054, 19242,    77,  2378,  9056,  1419,    80,     0]])

In [29]:
TEST_seq_o[-3:]

tensor([[ 7717, 18542,  7105,    77, 26313, 12434, 23496,  2866, 17241],
        [10291,  9056,    77, 13514, 22559,  2150,    77,  9054, 25836],
        [ 9054, 19242,    77,  2378,  9056,  1419,    80,     0,     0]])

In [30]:
TEST_T_seq_i = TEST_T_seq.narrow_copy(1, 0, TEST_T_seq.shape[1] - 1)
TEST_T_seq_o = TEST_T_seq.narrow_copy(1, 1, TEST_T_seq.shape[1] - 1)

In [31]:
TEST_T_seq_i[-3:]

tensor([[[15502],
         [ 7717],
         [18542],
         [ 7105],
         [   77],
         [26313],
         [12434],
         [23496],
         [ 2866]],

        [[10880],
         [10291],
         [ 9056],
         [   77],
         [13514],
         [22559],
         [ 2150],
         [   77],
         [ 9054]],

        [[   77],
         [ 9054],
         [19242],
         [   77],
         [ 2378],
         [ 9056],
         [ 1419],
         [   80],
         [    0]]])

In [32]:
TEST_T_seq_o[-3:]

tensor([[[ 7717],
         [18542],
         [ 7105],
         [   77],
         [26313],
         [12434],
         [23496],
         [ 2866],
         [17241]],

        [[10291],
         [ 9056],
         [   77],
         [13514],
         [22559],
         [ 2150],
         [   77],
         [ 9054],
         [25836]],

        [[ 9054],
         [19242],
         [   77],
         [ 2378],
         [ 9056],
         [ 1419],
         [   80],
         [    0],
         [    0]]])

In [39]:
TEST_seq_i.size()

torch.Size([23773, 9])

In [40]:
TEST_T_seq_i.size()

torch.Size([23773, 9, 1])

In [ ]:
data = torch.LongTensor(data)
# split tensor into tensors of of size time_steps
data = torch.split(tensor=data, split_size_or_sections=time_steps)

# note: word2index['<pad>'] = 0
sequences = pad_sequence(data, batch_first=True, padding_value=0)

# from seq we generate 2 copies.
# inputs=seq[:-1], targets=seq[1:]
sequences_inputs = sequences.narrow_copy(1, 0, sequences.shape[1] - 1)
sequences_targets = sequences.narrow_copy(1, 1, sequences.shape[1] - 1)

In [4]:
datasets = init_datasets(topic=topic, 
                         freq_threshold=freq_threshold, 
                         time_steps=time_steps, 
                         batch_size=batch_size, 
                         path=path)

In [5]:
vocab_size = datasets['vocab_size']
word2index = datasets['word2index']
data_loaders = datasets['data_loaders']

In [6]:
train, valid, test = data_loaders

In [23]:
train.__dict__

{'dataset': <data.Sequence_Data at 0x7f995dc6da90>,
 'num_workers': 0,
 'prefetch_factor': 2,
 'pin_memory': False,
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 20,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.RandomSampler at 0x7f995c43c670>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7f995c42c4f0>,
 'generator': None,
 'collate_fn': <function torch.utils.data._utils.collate.default_collate(batch)>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

In [24]:
x = next(iter(train))

In [29]:
x[0].size()

torch.Size([20, 9])

In [8]:
train.dataset.__dict__

{'x': tensor([[27455,  5710, 13303,  ...,   947,  1415,  1419],
         [   73, 14328,  1415,  ...,    80, 27455, 18542],
         [ 3600,   947,    74,  ..., 26313,  2866, 27455],
         ...,
         [14227, 23784,  3633,  ..., 25935,    77, 13024],
         [26051, 16618,  3633,  ..., 28195,  6893, 20420],
         [14227, 16618, 24915,  ...,  2341,  1654, 25804]]),
 'y': tensor([[ 5710, 13303, 23416,  ...,  1415,  1419,  5710],
         [14328,  1415, 28906,  ..., 27455, 18542, 26054],
         [  947,    74,    77,  ...,  2866, 27455,  5710],
         ...,
         [23784,  3633,  1419,  ...,    77, 13024, 15856],
         [16618,  3633,     2,  ...,  6893, 20420,    77],
         [16618, 24915,  3633,  ...,  1654, 25804,    80]]),
 'len': 202234}

In [14]:
train.batch_sampler.sampler.data_source.__dict__

{'x': tensor([[27455,  5710, 13303,  ...,   947,  1415,  1419],
         [   73, 14328,  1415,  ...,    80, 27455, 18542],
         [ 3600,   947,    74,  ..., 26313,  2866, 27455],
         ...,
         [14227, 23784,  3633,  ..., 25935,    77, 13024],
         [26051, 16618,  3633,  ..., 28195,  6893, 20420],
         [14227, 16618, 24915,  ...,  2341,  1654, 25804]]),
 'y': tensor([[ 5710, 13303, 23416,  ...,  1415,  1419,  5710],
         [14328,  1415, 28906,  ..., 27455, 18542, 26054],
         [  947,    74,    77,  ...,  2866, 27455,  5710],
         ...,
         [23784,  3633,  1419,  ...,    77, 13024, 15856],
         [16618,  3633,     2,  ...,  6893, 20420,    77],
         [16618, 24915,  3633,  ...,  1654, 25804,    80]]),
 'len': 202234}

In [10]:
train.dataset.x.size()

torch.Size([202234, 9])

In [20]:
train.dataset.x[0].view(-1, 1)

tensor([[27455],
        [ 5710],
        [13303],
        [23416],
        [18165],
        [27455],
        [  947],
        [ 1415],
        [ 1419]])